In [52]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
import pandas as pd

### Nous ferons ici une analyse économétrique des données comme vu dans les cours de l'ensae. 

In [53]:
file_path='Romain_Scrapping/offres_emploi_concatene_cleaned.csv'

# Nombre attendu de colonnes
expected_columns = 11

# Fonction personnalisée pour lire le fichier
def filter_csv(file_path, delimiter=','):
    with open(file_path, 'r') as file:
        filtered_rows = []
        for line in file:
            row = line.strip().split(delimiter)
            if len(row) == expected_columns:
                filtered_rows.append(row)
    return pd.DataFrame(filtered_rows)

# Chargement et affichage des données filtrées
data = filter_csv(file_path)

#Nomer les colonnes
data.columns = data.iloc[0]
data = data.drop(0).reset_index(drop=True)

### Regression sur les études et l'experiance

In [54]:
data_exp_etude = data[['salaire_annuel', 'etude_requise' , 'experience_requise']]

data_exp_etude['salaire_annuel'] = pd.to_numeric(data_exp_etude['salaire_annuel'], errors='coerce')
data_exp_etude['etude_requise'] = pd.to_numeric(data_exp_etude['etude_requise'], errors='coerce')
data_exp_etude['experience_requise'] = pd.to_numeric(data_exp_etude['experience_requise'], errors='coerce')

# Supprimer les lignes contenant des valeurs manquantes
data_exp_etude = data_exp_etude.dropna(axis=0, how='any')

/tmp/ipykernel_6842/1271999039.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_exp_etude['salaire_annuel'] = pd.to_numeric(data_exp_etude['salaire_annuel'], errors='coerce')
/tmp/ipykernel_6842/1271999039.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_exp_etude['etude_requise'] = pd.to_numeric(data_exp_etude['etude_requise'], errors='coerce')
/tmp/ipykernel_6842/1271999039.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

In [55]:
X = data_exp_etude[['etude_requise' , 'experience_requise']]
y = data_exp_etude[['salaire_annuel']]

# Initialisation et entraînement du modèle
model = LinearRegression()
model.fit(X, y)

X_with_const = sm.add_constant(X)  # Ajout de l'interception
model_stats = sm.OLS(y, X_with_const)  # Création du modèle
results = model_stats.fit()  # Ajustement du modèle

# Résumé détaillé
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:         salaire_annuel   R-squared:                       0.352
Model:                            OLS   Adj. R-squared:                  0.352
Method:                 Least Squares   F-statistic:                     3092.
Date:                Thu, 26 Dec 2024   Prob (F-statistic):               0.00
Time:                        08:05:58   Log-Likelihood:            -1.1969e+05
No. Observations:               11371   AIC:                         2.394e+05
Df Residuals:                   11368   BIC:                         2.394e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const               1.461e+04    262

### Regression par sécteur

In [56]:
categories = ['Informatique', 'Data', 'Marketing', 'Vente', 'Ressources_Humaines', 'Finance', 'Juridique', 'Ingenierie', 'Conseil', 'Education', 'Sante', 'Hotellerie', 'Media', 'Immobilier', 'E-commerce', 'Environnement', 'Agriculture', 'Transport']

data_secteur = data[['salaire_annuel']]
data_secteur['salaire_annuel'] = pd.to_numeric(data_secteur['salaire_annuel'], errors='coerce')

for categorie in categories:
    data_secteur[categorie] = 0

for i in range(len(data_secteur)):
    data_secteur.loc[i,data.loc[i,'source']] = 1

# Supprimer les lignes contenant des valeurs manquantes
data_secteur = data_secteur.dropna(axis=0, how='any')

/tmp/ipykernel_6842/1246982612.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_secteur['salaire_annuel'] = pd.to_numeric(data_secteur['salaire_annuel'], errors='coerce')
/tmp/ipykernel_6842/1246982612.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_secteur[categorie] = 0
/tmp/ipykernel_6842/1246982612.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

In [ ]:
X = data_secteur[categories].transpose()
y = data_secteur[['salaire_annuel']]
print(X)

# Initialisation et entraînement du modèle
model = LinearRegression()
model.fit(X, y)

X_with_const = sm.add_constant(X)  # Ajout de l'interception
model_stats = sm.OLS(y, X_with_const)  # Création du modèle
results = model_stats.fit()  # Ajustement du modèle

# Résumé détaillé
print(results.summary())

0      Informatique  Data  Marketing  Vente  Ressources_Humaines  Finance  \
0                 1     0          0      0                    0        0   
1                 1     0          0      0                    0        0   
2                 1     0          0      0                    0        0   
3                 1     0          0      0                    0        0   
4                 1     0          0      0                    0        0   
...             ...   ...        ...    ...                  ...      ...   
45244             0     0          0      0                    0        0   
45245             0     0          0      0                    0        0   
45246             0     0          0      0                    0        0   
45247             0     0          0      0                    0        0   
45248             0     0          0      0                    0        0   

0      Juridique  Ingenierie  Conseil  Education  Sante  Hotellerie  Media 